# Special Methods

Before we talk about the __init__() mehod in python. First we talk about the special method in python. The reason we want to have special methods in our classes is for use to design class that act like the native class that python provided, and that provide seamless interaction with Python. A good example is the for the class to act as a extensions of python.

Within the Python language, there are large number of special mehods names, and they fall into the following categories:
    1. Attribute Access: These special methods implement what we see as object.attribute in an expression, object.attribute on the left-hand side of an assignment, and object.attribute in a del statement
    2. Callables: This special method implements what we see as a function that is applied to arguents, much like the built in len() function
    3. Collections: These special methods implement the numerous features of collections. This involves methods like sequence[index], mapping[key], some_set|other_set
    4. Numbers: These special methods provide arithmetic operators and comparison operators. We can use these methods to expand the domain of numbers that python works with
    5. Context: There are two special mehods we'll use to implement a context manager that works with the with statement
    6. Iterators: There are special methods that define an iterator. This isn't essential since generator functions handle this feature so elegantly. However, we will take a look at how we can design our own iterators
   
## Chapter 1 starts

We will start talking about the most basic special methods == __init__(), this method permits a greate deal of latitude in providing the initial values for an object. In the case of an immutable object, this is the essential definition of the instance, and clarity becomes very important. In Chapter 1, we will review the numerous design alternatives for this method

Before we start talking about the __init__() mehod, we will first take a look the class hierarchy in Python, and start with the super class of all, the Object class

## The implicit superclass - Object

Object is a very simple class definition that does not include anything (almost). It is possible for us to create an Object, however, many of the special methods will not work on this class. In the meantime, all the class we make is a extend of the object class, for example:

In [8]:
class X:
    pass

class X(object):
    pass

print(X.__class__)
print(X.__class__.__base__)

<class 'type'>
<class 'object'>


This class is a sub-class of the super class object. As we can see from the type, a class is an object of the class named type, and the base class of the X object we created is the class named object.

### The base class object __init__() method

The fundamental to the life cycle of an object are its: 1. Creation, 2. Initialization, 3. destruction. Here we will focus on talking about the Initialization. 

The object class has a default implementation of __init__() that amounts to just pass. It is not a must to implement __init__() in our class, if we dont implement this __init__() then no instance variables will be created when the object is created, for some class we might not need __init__(). But most of the time, we will implement __init__() to create instance variable for our custom class

It is possible for us to add attributes to an object that is a subclass of the object. For example: 

In [13]:
class Rectangle:
    def area(self):
        return self.length * self.width
    
r = Rectangle()
r.length = 3
r.width = 4
print( "The area of the Rectangle instance is : {}".format(r.area()))

The area of the Rectangle instance is : 12


The above code is legal in Python, but it causes a lot confusion, thus, it is not considered as a good practice, which should be avoided. Though this could be useful in some cases, it is left to the developer to decide. An __init__() method should make the instance variables explict

### Implementing __init__() in a superclass

When an object is created, python first create an empty object, and then call the __init__() method for that object. The __init__() method generally creates the object's instance variables and performs any other one-time processing

In [15]:
class Card: # superClass Card
    def __init__(self, rank, suit):
        self.suit = suit
        self.rank = rank
        self.hard, self.sort = self._points()
        
class NumberCard(Card): # subClass NumberCard(Card)
    def _points(self):
        return int(self.rank), int(self.rank)

class AceCard(Card):
    def _points(self):
        return 1, 11

class FaceCard(Card):
    def _points(self):
        return 10, 10

In the example above, the __init__() method is in the superclass, so that a common initialization in the superclass, Card, applies to all the three subclasses. And we can also see the polymorphic design here, each sub class has their own implementation of the _points()_ method. All the subclasses have identical signatures, and all of them have the same signiture (a.k.a same attributes and methods). Thus objects of these three subclass can be used interchangeably

In [16]:
cards = [AceCard("A", "Heart"),NumberCard("2", "Heart"),NumberCard("3", "Heart")]

Here we created a list of cards that has different card in it. We enumerated the object into the list by hand. However, in the future, we will use factory function to create a list. And before we do that, we take a look at some other issues